In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
df = pd.DataFrame(iris['data'], columns=iris['feature_names'])
df['Target'] = iris['target']
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'Target']

In [4]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
Target          0
dtype: int64

In [5]:
df.duplicated().sum()

1

In [6]:
df.loc[df.duplicated(), :]

,sepal_length,sepal_width,petal_length,petal_width,Target
142,5.8,2.7,5.1,1.9,2


In [7]:
df.loc[(df.sepal_length==5.8) & (df.petal_width==1.9), :]

,sepal_length,sepal_width,petal_length,petal_width,Target
101,5.8,2.7,5.1,1.9,2
142,5.8,2.7,5.1,1.9,2


In [8]:
df = df.drop_duplicates()

In [9]:
df.loc[(df.sepal_length==5.8) & (df.petal_width==1.9), :]

,sepal_length,sepal_width,petal_length,petal_width,Target
101,5.8,2.7,5.1,1.9,2


In [10]:
from sklearn.model_selection import train_test_split

X_data = df.loc[:, 'sepal_length':'petal_width']
y_data = df.loc[:, 'Target']

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, 
                                                    shuffle=True, random_state=20)

# Hold-out 교차검정

In [12]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=True, random_state=20)

print(X_tr.shape, y_tr.shape)
print(X_val.shape, y_val.shape)


(83, 4) (83,)
(36, 4) (36,)


In [15]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=3, random_state=20)
rfc.fit(X_tr, y_tr)

y_tr_pred = rfc.predict(X_tr)
y_val_pred = rfc.predict(X_val)

tr_acc = accuracy_score(y_tr, y_tr_pred)
val_acc = accuracy_score(y_val, y_val_pred)
print(f"Train 정확도 : {tr_acc}")
print(f"Validation 정확도 : {val_acc}")

# Train 정확도는 높은 반면, Validation 정확도는 낮음 -> overfitting 의 가능성이 있다.

Train 정확도 : 0.9879518072289156
Validation 정확도 : 0.9166666666666666


In [17]:
y_test_pred = rfc.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)
print(f"Test 정확도 : {test_acc}")

# Test 예측시, 정확도가 낮아짐 -> Train data 에 overfitting 하기 때문에 새로운 데이터에 대한 예측력이 낮음.

Test 정확도 : 0.9


# K-fold 교차검증
: 이렇게 하는 이유는 데이터를 학습용/평가용 데이터 세트로 여러 번 나눈 것의 평균적인 성능을 계산하면, 한 번 나누어서 학습하는 것에 비해 일반화된 성능을 얻을 수 있기 때문이다.
-> 학습할 데이터가 부족한 경우, validation data 를 확보하면서 학습 데이터도 최대화하는 방법

In [23]:
from sklearn.model_selection import KFold

val_scores = []
num_fold = 1

for tr_idx, val_idx in KFold(n_splits=5).split(X_train, y_train):
  # 훈련용 데이터와 검증용 데이터를 행 인덱스 기준으로 추출
  X_tr, X_val = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
  y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]
  # 학습
  rfc = RandomForestClassifier(max_depth=5, random_state=20)
  rfc.fit(X_tr, y_tr)
  # 검증
  y_val_pred = rfc.predict(X_val)
  val_acc = accuracy_score(y_val, y_val_pred)
  print(num_fold, " 정확도 : ", val_acc)
  val_scores.append(val_acc)
  num_fold += 1

1  정확도 :  0.875
2  정확도 :  1.0
3  정확도 :  0.9166666666666666
4  정확도 :  0.9583333333333334
5  정확도 :  0.9565217391304348


## 평균 Accuracy 계산

In [24]:
import numpy as np

mean_score = np.mean(val_scores)

print(f"평균 : {np.round(mean_score, 4)}")

평균 : 0.9413
